In [1]:
import numpy as np
import pandas as pd

from keras import utils
from keras.models import Sequential
from keras.preprocessing import text
from keras.layers import Dense, Activation, Dropout

from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.
/Users/chaitu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chaitu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chaitu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chaitu/opt/anaconda3/lib/python3.7

In [2]:
df = pd.read_csv('tweet_cluster_mapping.csv')

df = df.sample(frac=1).reset_index(drop=True)
train_size = int(len(df) * .7)
train_posts = df['tweet'][:train_size]
train_tags = df['cluster_number'][:train_size]

test_posts = df['tweet'][train_size:]
test_tags = df['cluster_number'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts)  # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 20

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1924 samples, validate on 826 samples
Epoch 1/20
1924/1924 [==============================] - 1s 425us/step - loss: 1.8500 - accuracy: 0.2687 - val_loss: 1.7945 - val_accuracy: 0.3015
Epoch 2/20
1924/1924 [==============================] - 1s 323us/step - loss: 1.6458 - accuracy: 0.4054 - val_loss: 1.7073 - val_accuracy: 0.3584
Epoch 3/20
1924/1924 [==============================] - 1s 311us/step - loss: 1.4262 - accuracy: 0.5073 - val_loss: 1.6171 - val_accuracy: 0.3910
Epoch 4/20
1924/1924 [==============================] - 1s 317us/step - loss: 1.1745 - accuracy: 0.6320 - val_loss: 1.5650 - val_accuracy: 0.3947
Epoch 5/20
1924/1924 [==============================] - 1s 320us/step - loss: 0.9414 - accuracy: 0.7380 - val_loss: 1.5493 - val_accuracy: 0.4092
Epoch 6/20
1924/1924 [==============================] - 1s 340us/step - loss: 0.7538 - accuracy: 0.8035

In [3]:
# computing the accuracy score on the test data set
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

1179/1179 [==============================] - 0s 108us/step
Test accuracy: 0.4088210463523865
